In [ ]:
# Advanced Time Series Forecasting with Prophet and Hyperparameter Optimization
# Single-file Jupyter notebook style script that: 
# 1) programmatically generates a 3+ year hourly synthetic dataset with daily, weekly, yearly seasonality, trend shifts, and holidays
# 2) fits a baseline Prophet model
# 3) runs a systematic Grid Search over three Prophet hyperparameters using rolling-origin validation
# 4) trains final model and reports MAE, RMSE, MAPE and visualizations

# Notes:
# - Requires: pandas, numpy, matplotlib, prophet (or fbprophet), sklearn
# - If `prophet` import fails, try `fbprophet` (uncomment fallback)
# - This script is ready to paste into a Jupyter cell and run top-to-bottom.

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from itertools import product
import json
import os

# Try import Prophet
try:
    from prophet import Prophet
except Exception:
    try:
        from fbprophet import Prophet
    except Exception as e:
        raise ImportError('Prophet is required. Install with `pip install prophet` or `pip install fbprophet`.')

# -------------------------------
# 1) Synthetic data generation
# -------------------------------
SEED = 42
np.random.seed(SEED)

# Generate hourly date index for ~3.5 years (~3 years + buffer)
start = pd.to_datetime('2018-01-01')
periods = 24 * 3 * 365 + 24 * 180  # ~3.5 years of hourly data
date_index = pd.date_range(start=start, periods=periods, freq='H')

# Base components
def daily_seasonality(t, amplitude=10):
    # 24-hour cycle using sin
    return amplitude * np.sin(2 * np.pi * (t.hour / 24.0))

def weekly_seasonality(t, amplitude=5):
    # weekly pattern using day of week
    return amplitude * ((t.dayofweek >= 5) * 1.0)  # weekend bump

def yearly_seasonality(t, amplitude=3):
    # approximate yearly cycle using dayofyear
    return amplitude * np.sin(2 * np.pi * (t.dayofyear / 365.25))

# Convert to DataFrame
df = pd.DataFrame({'ds': date_index})

# Trend piecewise linear with changepoints (abrupt shifts)
# start with a base linear trend
base_trend = 0.001 * np.arange(len(df))

# Introduce random abrupt trend shifts at specified dates
changepoint_dates = [pd.Timestamp('2019-06-01'), pd.Timestamp('2020-09-15'), pd.Timestamp('2021-05-01')]
trend_shift = np.zeros(len(df))
for cp in changepoint_dates:
    idx = df.index[df['ds'] >= cp]
    if len(idx) > 0:
        # add a slope change after changepoint
        shift_amount = np.random.uniform(-0.0008, 0.0015)
        trend_shift[idx] += shift_amount * np.arange(len(idx))

trend = base_trend + np.concatenate(([0], np.cumsum(np.diff(base_trend) + np.concatenate((np.zeros(1), trend_shift[:-1])))))

# Build y from components
y = []
for t in df['ds']:
    val = 50.0  # base level
    val += daily_seasonality(t, amplitude=12)
    val += weekly_seasonality(t, amplitude=6)
    val += yearly_seasonality(t, amplitude=4)
    # add slowly changing trend (interpolate from computed trend)
    val += trend[df.index[df['ds'] == t][0]]
    y.append(val)

# Convert to numpy
y = np.array(y)

# Add holiday spikes/dips (predefined simulated holidays)
holidays = []
# We'll create a list of holiday dates and add effects across windows
holiday_list = [
    {'name': 'new_year', 'month': 1, 'day': 1},
    {'name': 'independence_day', 'month': 8, 'day': 15},
    {'name': 'diwali_approx', 'month': 11, 'day': 4},
    {'name': 'christmas', 'month': 12, 'day': 25}
]

# Simulate holiday effect as spike/dip in value for a small window
for year in range(start.year, start.year + 5):
    for h in holiday_list:
        try:
            d = pd.Timestamp(year=year, month=h['month'], day=h['day'])
        except Exception:
            continue
        # add a holiday effect for +/- 12 hours
        mask = (df['ds'] >= (d - pd.Timedelta(hours=12))) & (df['ds'] <= (d + pd.Timedelta(hours=12)))
        effect = np.random.uniform(-15, 20)  # holiday can be negative or positive
        y[mask.values] += effect * np.exp(-((np.arange(mask.sum()) - mask.sum()//2)**2) / (2*(4**2)))
        holidays.append({'ds': d, 'holiday': h['name']})

# Add heteroscedastic noise and small random outliers
noise = np.random.normal(scale=3.5 + 0.005 * np.arange(len(df)), size=len(df))
# occasional outliers
outlier_idx = np.random.choice(np.arange(len(df)), size=int(0.001 * len(df)), replace=False)
noise[outlier_idx] += np.random.choice([-40, 40], size=len(outlier_idx))

y = y + noise

# Compose final dataframe
df['y'] = y

# Sanity check
print('Generated data points:', len(df))
print(df.head())

# Save CSV
os.makedirs('/mnt/data', exist_ok=True)
csv_path = '/mnt/data/synthetic_hourly_timeseries.csv'
df.to_csv(csv_path, index=False)
print('Saved synthetic dataset to', csv_path)

# -------------------------------
# 2) Prepare Prophet-friendly data and holidays dataframe
# -------------------------------
# Build holidays DataFrame for Prophet
hol_df = pd.DataFrame(holidays)
if not hol_df.empty:
    hol_df = hol_df.drop_duplicates().reset_index(drop=True)
else:
    hol_df = pd.DataFrame(columns=['ds', 'holiday'])

# Prophet expects ds column as datetime and holiday names
if not hol_df.empty:
    hol_df['ds'] = pd.to_datetime(hol_df['ds'])

# -------------------------------
# 3) Train/test split (80/20)
# -------------------------------
split_idx = int(len(df) * 0.8)
train_df = df.iloc[:split_idx].copy()
test_df = df.iloc[split_idx:].copy()
print('Train length:', len(train_df), 'Test length:', len(test_df))

# -------------------------------
# Utility: Forecast evaluation metrics
# -------------------------------

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denom = np.where(np.abs(y_true) < 1e-8, 1e-8, np.abs(y_true))
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100

# -------------------------------
# 4) Baseline Prophet model
# -------------------------------
print('\nFitting baseline Prophet model...')
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True, holidays=hol_df, interval_width=0.95)
# set small changepoint_prior_scale as baseline
model.changepoint_prior_scale = 0.05
model.seasonality_prior_scale = 10.0
model.holidays_prior_scale = 10.0

model.fit(train_df[['ds', 'y']])

# Create future for test period
future = test_df[['ds']].copy()
forecast = model.predict(future)

# Evaluate baseline
baseline_pred = forecast['yhat'].values
base_mae = mean_absolute_error(test_df['y'].values, baseline_pred)
base_rmse = np.sqrt(mean_squared_error(test_df['y'].values, baseline_pred))
base_mape = mape(test_df['y'].values, baseline_pred)

print('Baseline MAE: {:.4f}, RMSE: {:.4f}, MAPE: {:.4f}%'.format(base_mae, base_rmse, base_mape))

# Plot baseline
plt.figure(figsize=(12,5))
plt.plot(test_df['ds'], test_df['y'], label='Actual', alpha=0.6)
plt.plot(test_df['ds'], baseline_pred, label='Baseline Prophet Forecast', alpha=0.8)
plt.legend()
plt.title('Baseline Forecast vs Actual (Test set)')
plt.tight_layout()
plt.show()

# -------------------------------
# 5) Grid Search with Rolling Origin Validation
# -------------------------------
print('\nStarting Grid Search with rolling-origin validation...')

# Parameter grid (concise but representative)
param_grid = {
    'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.5],
    'seasonality_prior_scale': [1.0, 5.0, 10.0],
    'holidays_prior_scale': [1.0, 5.0, 10.0]
}

# Rolling origin parameters
n_splits = 3  # number of rolling validations; can increase
min_train_size = int(len(train_df) * 0.6)
horizon = len(test_df)  # evaluate on last test horizon for final ranking

# Create split points (simple growing window)
split_points = []
start_idx = min_train_size
increment = int((len(train_df) - min_train_size) / (n_splits))
for i in range(n_splits):
    train_end = start_idx + i * increment
    if train_end + 24 > len(train_df):
        break
    split_points.append(train_end)

if not split_points:
    split_points = [min_train_size]

# Evaluate combinations
results = []
from sklearn.model_selection import ParameterGrid

for params in ParameterGrid(param_grid):
    scores = []
    for sp in split_points:
        train_split = train_df.iloc[:sp].copy()
        val_split = train_df.iloc[sp:sp + horizon].copy()
        if len(val_split) < 24:
            continue
        m = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True, holidays=hol_df, interval_width=0.95)
        m.changepoint_prior_scale = params['changepoint_prior_scale']
        m.seasonality_prior_scale = params['seasonality_prior_scale']
        m.holidays_prior_scale = params['holidays_prior_scale']
        try:
            m.fit(train_split[['ds', 'y']])
            # forecast validation horizon from val_split ds
            future_val = val_split[['ds']].copy()
            pred = m.predict(future_val)['yhat'].values
            true = val_split['y'].values
            rmse = np.sqrt(mean_squared_error(true, pred))
            scores.append(rmse)
        except Exception as e:
            scores.append(np.nan)
    # aggregate score
    if len(scores) == 0:
        mean_score = np.nan
    else:
        mean_score = np.nanmean(scores)
    results.append({'params': params, 'mean_rmse': mean_score})
    print('Tested:', params, '-> mean_rmse:', mean_score)

# Sort results
results_sorted = sorted(results, key=lambda x: (np.nan if x['mean_rmse'] is None else x['mean_rmse']))

# Show top 5
top_k = [r for r in results_sorted if not np.isnan(r['mean_rmse'])][:5]
print('\nTop parameter sets:')
for i, r in enumerate(top_k):
    print(i+1, r)

# Choose best
best = None
for r in results_sorted:
    if not np.isnan(r['mean_rmse']):
        best = r
        break

if best is None:
    # fallback to baseline hyperparams
    best_params = {'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'holidays_prior_scale': 10.0}
else:
    best_params = best['params']

print('\nSelected best_params:', best_params)

# -------------------------------
# 6) Train final model on full training set & forecast test set
# -------------------------------
print('\nTraining final model with best parameters on full training set...')
final_model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True, holidays=hol_df, interval_width=0.95)
final_model.changepoint_prior_scale = best_params['changepoint_prior_scale']
final_model.seasonality_prior_scale = best_params['seasonality_prior_scale']
final_model.holidays_prior_scale = best_params['holidays_prior_scale']

final_model.fit(train_df[['ds','y']])
future_test = test_df[['ds']].copy()
final_forecast = final_model.predict(future_test)

pred = final_forecast['yhat'].values
true = test_df['y'].values
final_mae = mean_absolute_error(true, pred)
final_rmse = np.sqrt(mean_squared_error(true, pred))
final_mape = mape(true, pred)

metrics = {
    'baseline': {'MAE': base_mae, 'RMSE': base_rmse, 'MAPE': base_mape},
    'final': {'MAE': final_mae, 'RMSE': final_rmse, 'MAPE': final_mape}
}

print('\nFinal metrics:')
print(json.dumps(metrics, indent=2))

# Save forecast and metrics
out_dir = '/mnt/data/prophet_project_outputs'
os.makedirs(out_dir, exist_ok=True)
final_forecast[['ds','yhat','yhat_lower','yhat_upper']].to_csv(os.path.join(out_dir, 'final_forecast.csv'), index=False)
with open(os.path.join(out_dir, 'metrics.json'), 'w') as f:
    json.dump(metrics, f)
print('Saved outputs to', out_dir)

# -------------------------------
# Plots for final model
# -------------------------------
plt.figure(figsize=(14,5))
plt.plot(test_df['ds'], true, label='Actual', alpha=0.6)
plt.plot(test_df['ds'], pred, label='Final Prophet Forecast', alpha=0.9)
plt.fill_between(test_df['ds'], final_forecast['yhat_lower'], final_forecast['yhat_upper'], alpha=0.2)
plt.legend()
plt.title('Final Forecast vs Actual (Test set)')
plt.tight_layout()
plt.show()

# Plot components
fig = final_model.plot_components(final_forecast)
plt.tight_layout()
plt.show()

# Summary table
summary_df = pd.DataFrame([{
    'model': 'baseline',
    'MAE': base_mae,
    'RMSE': base_rmse,
    'MAPE': base_mape
}, {
    'model': 'final',
    'MAE': final_mae,
    'RMSE': final_rmse,
    'MAPE': final_mape
}])

print('\nPerformance summary:')
print(summary_df)

# Save summary
summary_df.to_csv(os.path.join(out_dir, 'summary_metrics.csv'), index=False)

# End of notebook
print('\nNotebook run complete. Files saved in /mnt/data and /mnt/data/prophet_project_outputs')
